## Fibonacci, several ways
First, we define the Fibonacci sequence.
$x_0 = 1$, $x_1 = 1$, and for any $n > 1$, $x_n = x_{n - 1} + x_{n - 2}$.

The first few values, then, are $1, 1, 2, 3, 5, 8, 13, 21 ...$

They appear everywhere in nature! See this list for a few examples: https://science.howstuffworks.com/math-concepts/fibonacci-nature.htm

There's a number of ways to calculate it.

In [1]:
def fibonacci_recursive(n):

    # only defined for n > 0
    assert(n >= 0)

    # if we want x_0
    if n == 0:
        return 1

    # if we want x_1
    if n == 1:
        return 1

    # if we want any other x_n
    return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

In [2]:
for i in range(10):
    print(fibonacci_recursive(i))

1
1
2
3
5
8
13
21
34
55


This is an interesting example, but for even moderately large n, it is quite slow. That's because it keeps computing the same values over and over again.

In [3]:
import time
begin = time.time()
fibonacci_recursive(40)
end = time.time()
print('Time elapsed (seconds): ', end-begin)

Time elapsed (seconds):  42.334558963775635


This takes a long time. How can we speed it up?
One way is to store values we already used. This turns our "recursive" function above into something called "dynamic programming"

In [4]:
def fibonacci_dynamic(n):
    assert n >= 0
    if n == 0:
        return 0
    if n == 1:
        return 1
    numbers = [0, 1]

    while len(numbers) <= n:
        # next number = last two numbers in the list
        next_number = numbers[-1] + numbers[-2]

        # add it to the list
        numbers.append(next_number)

    # return the last number in the list
    return numbers[-1]

In [5]:
for i in range(10):
    print(fibonacci_dynamic(i))

0
1
1
2
3
5
8
13
21
34


How long does it take?

In [6]:
import time
begin = time.time()
fibonacci_dynamic(40)
end = time.time()
print('Time elapsed (seconds): ', end-begin)
# almost immediate

Time elapsed (seconds):  8.702278137207031e-05


## More advanced:

We can even approximate it. To see how we get this approximation, first note something. Let's look at a list of Fibonacci numbers, and see how they grow




In [7]:
def fibonacci_dynamic_list(n):
    numbers = [0, 1]    
    while len(numbers) <= n:
        next_number = numbers[-1] + numbers[-2]
        numbers.append(next_number)
    return numbers
fibonacci_numbers = fibonacci_dynamic_list(15)
fibonacci_numbers

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

In [8]:
# Check the ratio of one fibonacci number to the previous one:
ratios = []
for i in range(2, 15):
    ratios.append(fibonacci_numbers[i]/fibonacci_numbers[i - 1])
ratios

[1.0,
 2.0,
 1.5,
 1.6666666666666667,
 1.6,
 1.625,
 1.6153846153846154,
 1.619047619047619,
 1.6176470588235294,
 1.6181818181818182,
 1.6179775280898876,
 1.6180555555555556,
 1.6180257510729614]

It converges to $\phi$ (pronounced "Fee" in Greece) the Golden Ratio. This is seen many places in nature; see this list for example: https://gizmodo.com/15-uncanny-examples-of-the-golden-ratio-in-nature-5985588

How can we show it actually converges? The math is more straightforward than one would expect, but it takes a little setup. Consider the following system of equations:
$$x_{n - 1} + x_{n - 2} = x_n$$


How can we show it actually converges? The math is more straightforward than one would expect, but it takes a little setup. Consider the following equation:

$$x_{n - 1} + x_{n - 2} = x_n$$

Suppose we think there is some number $\lambda$ such that for every $n$, $x_{n} = \lambda x_{n - 1}$. Then, also, $x_n = \lambda^2 x_{n - 2}$. We can rewrite the above equation as

$$\lambda x_{n - 2} + x_{n - 2} = \lambda^2 x_{n - 2}$$

Or, getting rid of the cumbersome $x_{n - 2}$,

$$\lambda z + z = \lambda^2 z$$

Assuming $z > 0$ (which it is for all fibonacci numbers, besides the first), divide by $z$ to get

$$\lambda  + 1 = \lambda^2$$

or 

$$ \lambda^2 - \lambda - 1 = 0$$

A standard quadratic equation. If we remember how to solve those...

In [9]:
def quadratic_solutions(a, b, c):
    d = (b ** 2 - 4 * a * c) ** 0.5
    r1 = (-b + d)/(2 * a)
    r2 = (-b - d)/(2 * a)
    return r1, r2

quadratic_solutions(1, -1, -1)

(1.618033988749895, -0.6180339887498949)

The first number should look familiar... that's $\phi$!
The second one is important too.

Note that if we actually solved that by hand, we would get the solutions as
$$\frac{1 \pm \sqrt{5}}{2}$$

so $\phi = \frac{1 + \sqrt{5}}{2}$.

How can we actually use this? Any solution that works like the above needs to be a combination of the two solutions we have. And any combination of the two solutions works. Call the two solutions $\lambda_+$ and $\lambda_-$. 

So we don't have a perfect match for Fibonacci yet. If you just try to do what we have with phi, it doesn't match our numbers. We get:

In [10]:
phi = (1 + 5 ** 0.5)/2
for i in range(10):
    print(phi ** i)

1.0
1.618033988749895
2.618033988749895
4.23606797749979
6.854101966249686
11.090169943749476
17.944271909999163
29.03444185374864
46.978713763747805
76.01315561749645


Which is not correct. But consider the following trick:
Let's define a few new sequences:
$$y_{n} = y_{n - 1} + y_{n - 2}$$
$$z_{n} = z_{n - 1} + z_{n - 2}$$
$$w_{n} = y_{n} + z_{n}$$

So far $y$ and $z$ looks like fibonacci. What about $w$?

$$w_{n} = y_{n} + z_{n}$$
$$= y_{n - 1} + y_{n - 2} + z_{n - 1} + z_{n - 2}$$
$$ = (y_{n - 1} + z_{n - 1}) + (y_{n - 2} + z_{n - 2})$$
$$ w_{n} = w_{n - 1} + w_{n - 2}$$
 
Which has the same properties. Great! $y$ and $z$ are independent, but we still get this property! What that allows us to do is use the values we had above, and try to recover our Fibonacci sequnce. In particular, let's let

$$y_{n} = \lambda_+ y_{n - 1}$$
$$z_{n} = \lambda_- z_{n - 1}$$

and try to find $y_0, z_0$ so that we get Fibonacci back. 

Note that those imply $$y_n = \lambda_+^n y_0$$ (and similarly for $z_n$)

In particular, we can recover the Fibonacci sequence by solving for its first two values.

$$y_0 + z_0 = 0$$
$$\lambda_+ y_0 + \lambda_- z_0 = 1$$

Then, $$y_0 = -z_0$$.

$$y_0 \frac{1 + \sqrt{5}}{2} - y_0\frac{1 - \sqrt{5}}{2} = 1$$
$$y_0\sqrt{5} = 1$$
$$y_0 = \frac{1}{\sqrt{5}}, z_0 =  \frac{-1}{\sqrt{5}}$$

And finally, this means that the nth Fibonacci number is given by:

$$x_n = \frac{1}{\sqrt{5}}\Big(\frac{1 + \sqrt{5}}{2}\Big)^n - \frac{1}{\sqrt{5}}\Big(\frac{1 - \sqrt{5}}{2}\Big)^n$$

Note that the second term gets small because $(\frac{1 - \sqrt{5}}{2})$ is between -1 and 1, so it shrinks to zero as we raise the exponent. That's why the ratio of the Fibonacci numbers approaches $\phi$!!

In [11]:
import math
def fibonacci_closed_form(n):
    # Answer appears slightly off due to computers having to round numbers.
    s = math.sqrt(5)
    r1 = (1 + s)/2
    r2 = (1 - s)/2
    return (1/s) * r1 ** n - (1/s) * r2 ** n
for i in range(10):
    print(fibonacci_closed_form(i))

0.0
1.0
0.9999999999999999
2.0
3.0000000000000004
5.000000000000001
8.000000000000002
13.000000000000004
21.000000000000004
34.000000000000014


## Even more advanced:
How else can we get these?

Let's return to the equation we had before:

$$x_{n - 1} + x_{n - 2} = x_n$$


If we add another equation, it becomes clear that we can set this up as a system of linear equations.

$$x_{n - 1} + x_{n - 2} = x_n$$
$$x_{n - 1}  = x_{n - 1}$$

Putting this in matrix-vector notation:

$$ \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix} \begin{bmatrix} x_{n - 1} \\ x_{n - 2} \end{bmatrix} = \begin{bmatrix} x_n \\ x_{n - 1} \end{bmatrix}$$

Or, rewriting with the notation $v_n =  \begin{bmatrix} x_n \\ x_{n - 1} \end{bmatrix}$ and $A = \begin{bmatrix} 1 & 1 \\ 1 & 0 \end{bmatrix}$

$$Av_n = v_{n + 1}$$


In [ ]:
import numpy as np
A = np.array([
    [1, 1],
    [1, 0]
])
v = np.array([1, 0])
A @ v

In [ ]:
A @ A @ v

In [ ]:
A @ A @ A @ v

In [15]:
A @ A @ A @ A @ v

array([5, 3])

An Eigenvector of a matrix is a vector such that $Av = \lambda v$ -- in other words, it becomes just a multiple of the previous vector. That multiple $\lambda$ is known as an eigenvaule. When a matrix has eigenvalues and eigenvectors, we can easily see what happens when we multiply that vector with that matrix many times.
$A(Av) = \lambda^2v, A(A(Av))= \lambda^3v$, etc.

$A$ is symmetric and invertible, and so it has two non-zero eigenvalues. (This fact is not obvious; the proof itself is simple but to understand the setup you need a few years of university-level mathematics.) Any guesses what they are?

In [16]:
eigenvalues, eigenvectors = np.linalg.eig(A)
print('Eigenvalues:', eigenvalues)

Eigenvalues: [ 1.61803399 -0.61803399]


$A$ also has two eigenvectors (one for each eigenvalue.) We can write any vector as a combination of these vectors; specifically, we can write the vector $v_1 = \begin{bmatrix} 1 \\  0 \end{bmatrix}$ as them. (Let's let the computer do this. It's doable by hand but annoying, especially to type up.)

In [17]:
#sanity check on the eigenvector thing:
vector_1 = eigenvectors[:, 0]
vector_2 = eigenvectors[:, 1]
lambda_1, lambda_2 = eigenvalues

print('vector 1:', vector_1)
print('vector 2:', vector_2)

vector 1: [0.85065081 0.52573111]
vector 2: [-0.52573111  0.85065081]


In [18]:
#sanity check on the eigenvector thing:
A @ vector_1

array([1.37638192, 0.85065081])

In [19]:
# Check that this is the same as above!
lambda_1 * vector_1

array([1.37638192, 0.85065081])

In [20]:
# Solve the initial vector
b = np.array([1, 0])
c1, c2 = np.linalg.inv(eigenvectors) @ b

In [21]:
c1 * vector_1 + c2 * vector_2

array([1., 0.])

What did we get? We got the initial condition as a combination of the eigenvectors. Let's call this $v_1 = c_1x_1 + c_2x_2$ The value of that is this:

$$v_n = A^nv_1 = A^n(c_1x_1 + c_2x_2)$$
$$ = A^n(c_1x_1) + A^n(c_2x_2)$$
$$ = c_1\lambda_1^nx_1 + c_2\lambda_2^nx_2$$

where $\lambda_1 \approx 1.618$ and $\lambda_2 \approx -0.618$

And note that as $n$ gets large, $\lambda_1^n$ gets large and $\lambda_2^n$ gets small. This is another argument showing that the ratio of subsequent fibonacci numbers converges to $\phi$
